In [1]:
from scipy.integrate import solve_ivp
from numba import njit
import matplotlib.pyplot as plt
import numpy as np
from timeit import timeit
from time import time
from statistics import mean, stdev
import simanneal
from sklearn.metrics import mean_squared_error
from scipy.optimize import least_squares

In [2]:
@njit
def fun_sir_lagrange_numba(t,y,Out, In, Beta, Gamma):
    
    K = Out.shape[0]
    Out_i_k = Out.sum(axis=1)

    y = y.reshape((4,K,K))
    I_k_i = y[1].sum(axis=0)
    N_k_i = y[3].sum(axis=0)
    new_y = np.zeros((4,K,K))
    for i in range(K):
        for j in range(K):
            if i == j:
                new_y[0,i,i] = - Beta[i] * y[0,i,i] * I_k_i[i] / N_k_i[i] - \
                                y[0,i,i] * Out_i_k[i] + (In[i,] * y[0,i]).sum()
                new_y[1,i,i] =   Beta[i] * y[0,i,i] * I_k_i[i] / N_k_i[i] - Gamma[i] * y[1,i,i] - \
                                y[1,i,i] * Out_i_k[i] + (In[i,] * y[1,i]).sum()
                new_y[2,i,i] =   Gamma[i] * y[1,i,i] - \
                                y[2,i,i] * Out_i_k[i] + (In[i,] * y[2,i]).sum()
                new_y[3,i,i] = - y[3,i,i] * Out_i_k[i] + (In[i,] * y[3,i]).sum()
            else:
                new_y[0,i,j] = - Beta[j] * y[0,i,j] * I_k_i[j] / N_k_i[j] - \
                                In[i,j] * y[0,i,j] + Out[i,j] * y[0,i,i]
                new_y[1,i,j] =   Beta[j] * y[0,i,j] * I_k_i[j] / N_k_i[j] - Gamma[j] * y[1,i,j] - \
                                In[i,j] * y[1,i,j] + Out[i,j] * y[1,i,i]
                new_y[2,i,j] =   Gamma[j] * y[1,i,j] - \
                                In[i,j] * y[2,i,j] + Out[i,j] * y[2,i,i]
                new_y[3,i,j] = - In[i,j] * y[3,i,j] + Out[i,j] * y[3,i,i]

    new_y = new_y.reshape((4*K*K,))
    return new_y

In [162]:
Beta = np.array([0.25, 0.25])
Gamma = np.array([0.052,0.052])
Out = np.array([
    [0, 0.5],
    [0.5,0]
])
In = Out.copy()
y0 = np.zeros((4,2,2))
np.fill_diagonal(y0[0], 999)
np.fill_diagonal(y0[1], 1)
np.fill_diagonal(y0[3], 1000)
y0 = y0.flatten()
ts = np.linspace(0,300,301)

real_sol = solve_ivp(fun_sir_lagrange_numba,(0,300), y0, t_eval=ts, args=(Out, In, Beta, Gamma))
real_y = real_sol.y.reshape((4, 2,2, real_sol.y.shape[1]))
real_I = real_y[1].sum(axis=1)
real_I = real_I.flatten()

print(real_I.shape)

def fitness(value):
    new_Beta = np.array([value[0], value[0]])
    new_Gamma = np.array([value[1], value[1]])
    #new_Out = np.array([
    #    [0, value[4]],
    #    [value[5],0]
    #])
    #new_In = np.array([
    #    [0, value[6]],
    #    [value[7],0]
    #])
    estimate_sol = solve_ivp(fun_sir_lagrange_numba,(0,300), y0, t_eval=ts, args=(Out, In, new_Beta, new_Gamma))
    estimate_y = estimate_sol.y.reshape((4, 2,2, estimate_sol.y.shape[1]))
    estimate_I = estimate_y[1].sum(axis=1)
    estimate_I = estimate_I.flatten()
    #er =  mean_squared_error(real_I, estimate_I)
    return mean_squared_error(real_I, estimate_I)
    return real_I - estimate_I
    

(602,)


In [5]:
import simanneal
import random

class MySA(simanneal.Annealer):
    def move(self):
        x, y = self.state
        x_new = x + (0.5 - random.random())  # Randomly perturb x
        x_new = max(x_new, 1e-4)

        y_new = y + (0.5 - random.random())  # Randomly perturb y
        y_new = max(y_new, 1e-4)
        
        self.state = (x_new, y_new)

    def energy(self):
        return fitness(self.state)
        

# Initial state
initial_state = (1.0, 1.0)

# Create an instance of the annealer
sa = MySA(initial_state)

# Set the parameters for the annealing process
sa.set_schedule(sa.auto(minutes=1))

# Perform the annealing
state, e = sa.anneal()

print("Optimal state:", state)
print("Minimum energy:", e)


 Temperature        Energy    Accept   Improve     Elapsed   Remaining
    66.00000        216.90     0.05%     0.00%     0:01:56     0:22:15

In [14]:
from timeit import timeit
import random

In [151]:
r = least_squares(fitness, x0= np.array([random.random(),random.random(),random.random(),random.random()]), bounds=(0,1), verbose=1)
print(r.x)
print(r.cost)
print(r.optimality)

`xtol` termination condition is satisfied.
Function evaluations 86, initial cost 5.8138e+06, final cost 2.3460e-25, first-order optimality 1.18e-08.
[0.25  0.25  0.052 0.052]
2.3459722803168234e-25
1.1771622233312537e-08


In [154]:
import pyswarm

# Define la función de costo que queremos minimizar
def objective_function(params):
    x, y = params
    return x**2 + y**2

# Llama a la función de optimización de enjambre de partículas
x_opt, y_opt = pyswarm.pso(fitness, lb=[0.0, 0.0,0.0, 0.0], ub=[1, 1,1, 1],swarmsize=40, maxiter=100, debug=True)

print("Optimal x:", x_opt)
print("Optimal y:", y_opt)
print("Minimum value:", fitness(x_opt))


No constraints given.
New best for swarm at iteration 1: [0.48104744 0.46083275 0.09718038 0.68005328] 15085.98607394335
New best for swarm at iteration 1: [0.32042845 0.54566321 0.23218832 0.18541053] 13734.030558488703
New best for swarm at iteration 1: [0.65393888 0.18172047 0.20879289 0.09072557] 13373.1790129557
Best after iteration 1: [0.65393888 0.18172047 0.20879289 0.09072557] 13373.1790129557
New best for swarm at iteration 2: [0.68550063 0.01826015 0.24613134 0.        ] 9528.108726247106
Best after iteration 2: [0.68550063 0.01826015 0.24613134 0.        ] 9528.108726247106
New best for swarm at iteration 3: [0.35594249 0.28809518 0.13308701 0.07515605] 5704.224403467264
New best for swarm at iteration 3: [0.21676803 0.26382832 0.08027295 0.        ] 1504.8934301981224
Best after iteration 3: [0.21676803 0.26382832 0.08027295 0.        ] 1504.8934301981224
New best for swarm at iteration 4: [0.22022838 0.24815808 0.09836775 0.        ] 395.6625295584415
Best after iteration

In [189]:
import numpy as np
from scipy.optimize import differential_evolution
#import sko.PSO



# Define la función de costo que queremos minimizar
def objective_function(params):
    x, y = params
    return x**2 + y**2

# Definir los límites de búsqueda para x e y
bounds = [(-10, 10), (-10, 10)]

my = np.zeros((2,))

def fitness_DE(value):
    new_Beta = np.array([value[0], value[0]])
    new_Gamma = np.array([value[1], value[1]])
    estimate_sol = solve_ivp(fun_sir_lagrange_numba,(0,300), y0, t_eval=ts, args=(Out, In, new_Beta, new_Gamma))
    estimate_y = estimate_sol.y.reshape((4, 2*2, estimate_sol.y.shape[1]))
    estimate_I = estimate_y[1].sum(axis=0)
    return mean_squared_error(real_I, estimate_I)

def io(xk, convergence):
    if fitness_DE(xk) < 10:
        print('DATANTA')
        return True
    return False

print(io(1234, 566))

# Realizar la optimización con Evolución Diferencial
result = differential_evolution(fitness_DE, [(0, 1), (0, 1)],
                                maxiter=10,atol=0.01,disp=True,polish=True,
                                workers=1, callback=io)
print(result.items())
print("Optimal parameters:", result.x)
print("Minimum value:", result.fun)
my

TypeError: 'int' object is not subscriptable

In [186]:
help(differential_evolution)

Help on function differential_evolution in module scipy.optimize._differentialevolution:

differential_evolution(func, bounds, args=(), strategy='best1bin', maxiter=1000, popsize=15, tol=0.01, mutation=(0.5, 1), recombination=0.7, seed=None, callback=None, disp=False, polish=True, init='latinhypercube', atol=0, updating='immediate', workers=1, constraints=(), x0=None, *, integrality=None, vectorized=False)
    Finds the global minimum of a multivariate function.
    
    The differential evolution method [1]_ is stochastic in nature. It does
    not use gradient methods to find the minimum, and can search large areas
    of candidate space, but often requires larger numbers of function
    evaluations than conventional gradient-based techniques.
    
    The algorithm is due to Storn and Price [2]_.
    
    Parameters
    ----------
    func : callable
        The objective function to be minimized. Must be in the form
        ``f(x, *args)``, where ``x`` is the argument in the form o